In [1]:
import utils

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy

In [2]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [3]:
train = pd.read_csv('data/train_data_complaints_repeats_doctors.csv')

In [4]:
train.head(3)

,Unnamed: 0,Id_Записи,Id_Пациента,Возраст,Диагноз,Жалобы,Источник_рекламы,Клиника,Код_диагноза,Пол,Услуга,Жалобы (ngramm),Жалобы (unigramm),Врач,Повторный приём
0,0,0,115819,54,Гипертензивная болезнь сердца [гипертоническая...,"на повышение ад утром до 140/90 мм.рт.ст., пер...",Другое,5,I11,2,"Прием врача-кардиолога повторный, амбулаторный",повышение_ада,повышение ада утром мм рт ст периодич головокр...,кардиолог,1
1,1,1,399973,32,Доброкачественное новообразование молочной железы,На наличие опухоли в левой молочной железе,Другое,3,D24,2,"Прием врача-онколога (маммолога), повторный, а...",наличие_опухоль левый_молочный_железо,наличие опухоль левый молочный железо,онколог,1
2,2,2,427563,72,Простой хронический бронхит,Активных жалоб нет.,Интернет,6,J41.0,2,Прием первичный врача-пульмонолога,активный_жалоба,активный жалоба,пульмонолог,0


In [5]:
def preproces_fit(data):
    # Complaints
    complaints = data['Жалобы (unigramm)']
    tfidf_complaints = TfidfVectorizer(ngram_range=(1,2), min_df=10, stop_words=stopwords.words('russian'))
    tfidf_complaints.fit(complaints)
    
    # Doctors
    doctors = train['Врач'].fillna('sss')
    doctors_voc, counts = np.unique(doctors, return_counts=True)
    pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
    doctors[doctors == 'sss'] = pop_doctor
    
    vect_doctors = CountVectorizer()
    vect_doctors.fit(doctors)
    
    # Classes
    pop_diagnoses = set(utils.get_most_popular_diagnoses(data, percent=.8))
    most_pop_diagnose = scipy.stats.mode(data['Код_диагноза'])[0][0]
    y = data['Код_диагноза'].apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return tfidf_complaints, vect_doctors, pop_doctor, y

def preprocess_transform(tfidf_complaints, vect_doctors, pop_doctor, data):
    # Complaints
    complaints = data['Жалобы (unigramm)']
    
    # Doctors
    doctors = data['Врач'].fillna(pop_doctor)
    
    # Gender
    gender = data['Пол'].copy()
    gender[data['Пол'] == 1] = 0
    gender[data['Пол'] == 2] = 1
    
    # Repeats
    repeats = data['Повторный приём']
    
    # Age
    age = data['Возраст']
    
    print(doctors.shape, gender.shape, repeats.shape, age.shape)
    
    return np.hstack([
        tfidf_complaints.transform(complaints).todense(),
        vect_doctors.transform(doctors).todense(),
        np.expand_dims(gender, axis=1),
        np.expand_dims(repeats, axis=1),
        np.expand_dims(age, axis=1)
    ])

In [6]:
class DoctorsPopularityTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        doctors = x.fillna('sss')
        doctors_voc, counts = np.unique(doctors, return_counts=True)
        self.pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
        
        return self

    def transform(self, x):
        x = x.fillna('sss')
        x[x == 'sss'] = self.pop_doctor
        
        return x
    
class GenderTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = x.copy()
        x[x == 1] = 0
        x[x == 2] = 1
        
        return np.expand_dims(x, axis=1)
    
class AgeTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.expand_dims(x, axis=1)

In [7]:
def preproc_target_train(data):
    diagnoses = data['Код_диагноза'].copy()
    
    pop_diagnoses = set(utils.get_most_popular_diagnoses(diagnoses, percent=.80))
    most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def preproc_target_test(data, pop_diagnoses, most_pop_diagnose):
    diagnoses = data['Код_диагноза'].copy()
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

In [16]:
pipe = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('complaints_pipe', Pipeline([
                ('complaint_selector', ItemSelector(key='Жалобы (unigramm)')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=10, stop_words=stopwords.words('russian')))
            ])),
            ('doctor_pipe', Pipeline([
                ('doctor_selector', ItemSelector(key='Врач')),
                ('doc_pop', DoctorsPopularityTransformator()),
                ('count_vect', CountVectorizer())
            ])),
            ('gender_pipe', Pipeline([
                ('gender_selector', ItemSelector(key='Пол')),
                ('gender_transform', GenderTransformator())
            ])),
            ('age_pipe', Pipeline([
                ('age_selector', ItemSelector(key='Возраст')),
                ('age_transformator', AgeTransformator())
            ]))
        ]
    )),
    ('clf', SVC(C=0.1))
])

In [17]:
train, valid = train_test_split(train, test_size=0.3)

In [18]:
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train)
valid_y, _, _ = preproc_target_test(valid, pop_diagnoses, most_pop_diagnose)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [20]:
pipe.fit(train, train_y)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('complaints_pipe', Pipeline(memory=None,
     steps=[('complaint_selector', ItemSelector(key='Жалобы (unigramm)')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, e...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [34]:
test = pd.read_csv('data/test_data_complaints_repeats_doctors.csv')

In [52]:
pred = pipe.predict(test)

In [56]:
np.unique(pred)

array(['I11', 'J06.9', 'N76.0'], dtype=object)